<a href="https://colab.research.google.com/github/mahault/Multi-agent-sustainability/blob/main/Multiagent_Sustainability_toy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment Setup


*   Grid Setup: The environment consists of a 3x3 grid.
*   Agent Dynamics: Two agents that can move, communicate, and consume resources.
*   Resource Dynamics: Water and food are placed randomly and can deplete and replenish.




In [145]:
!pip install pettingzoo

In [156]:
import torch
import torch.nn.functional as F
from torch import nn
import torch.optim as optim
import torch.distributions as dist
import numpy as np
import random
from collections import deque
from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector
from gym import spaces

In [157]:
class Actor(nn.Module):
    def __init__(self, obs_size, action_size):
        super(Actor, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, 128),
            nn.ReLU(),
            nn.Linear(128, action_size),
            nn.Softmax(dim=-1)  # Action probabilities
        )

    def forward(self, x):
        return self.net(x)

class Critic(nn.Module):
    def __init__(self, obs_size, action_size):
        super(Critic, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size + action_size, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x, actions):
        return self.net(torch.cat([x, actions], dim=1))

In [158]:
class MADDPGAgent:
    def __init__(self, obs_size, action_size, n_agents):
        self.actor = Actor(obs_size, action_size).float()
        self.critic = Critic(obs_size * n_agents, action_size * n_agents).float()

        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=0.01)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=0.01)

        self.target_actor = Actor(obs_size, action_size).float()
        self.target_critic = Critic(obs_size * n_agents, action_size * n_agents).float()

        self.target_actor.load_state_dict(self.actor.state_dict())
        self.target_critic.load_state_dict(self.critic.state_dict())

    def act(self, state):
        grid_state = torch.FloatTensor(state["grid"].flatten()).unsqueeze(0)
        action_probs = self.actor(grid_state)
        distribution = dist.Categorical(action_probs)
        action = distribution.sample().item()
        return action

    def update(self, batch, agent_id, agents, gamma=0.99, tau=0.01):

        states, actions, rewards, next_states, dones = batch

        # Get the number of agents
        n_agents = len(agents)

        # Get the current agent's state and action
        state = states[agent_id]
        action = actions[agent_id]

        # Get the next state for the current agent
        next_state = next_states[agent_id]

        # Get the target actions for the next states using the target actor networks
        next_actions = []
        for i in range(n_agents):
            next_action = agents[i].target_actor(next_states[i])
            next_actions.append(next_action)
        next_actions = torch.cat(next_actions, dim=1)

        # Compute the target Q-value using the target critic network
        target_q = agents[agent_id].target_critic(next_state, next_actions)
        target_q = rewards[agent_id] + (gamma * target_q * (1 - dones[agent_id]))

        # Compute the current Q-value using the critic network
        current_q = agents[agent_id].critic(state, actions)

        # Compute the critic loss using the TD error (Q-targets - Q-values)
        critic_loss = F.mse_loss(current_q, target_q)

        # Update the critic network by minimizing the critic loss
        agents[agent_id].critic_optimizer.zero_grad()
        critic_loss.backward()
        agents[agent_id].critic_optimizer.step()

        # Compute the actor loss using the critic network and the current actions
        actor_loss = -agents[agent_id].critic(state, actions).mean()

        # Update the actor network by minimizing the actor loss
        agents[agent_id].actor_optimizer.zero_grad()
        actor_loss.backward()
        agents[agent_id].actor_optimizer.step()

        # Perform soft updates of the target networks for both actor and critic
        for target_param, param in zip(agents[agent_id].target_actor.parameters(), agents[agent_id].actor.parameters()):
            target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)

        for target_param, param in zip(agents[agent_id].target_critic.parameters(), agents[agent_id].critic.parameters()):
            target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)


In [147]:
class MADDPGAgent:
    def __init__(self, obs_size, action_size, n_agents):
        self.actor = Actor(obs_size, action_size).float()  # Ensure the model is in float
        self.critic = Critic(obs_size * n_agents, action_size * n_agents).float()  # Adjust according to the total number of agents

        self.position = (0, 0)  # Example initial position
        self.water_timer = 3
        self.food_timer = 7
        # Initialize beliefs, etc.
        # Assume beliefs and other state variables are initialized similarly

        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=0.01)  # Example learning rate
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=0.01)  # Example learning rate

        self.target_actor = Actor(obs_size, action_size).float()
        self.target_critic = Critic(obs_size * n_agents, action_size * n_agents).float()

        # Initialize target networks with the same weights as the original networks
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.target_critic.load_state_dict(self.critic.state_dict())

    def act(self, state):
      grid_state = state["grid"]
      grid_state = torch.FloatTensor(grid_state.flatten()).unsqueeze(0)
      action_probs = self.actor(grid_state)
      action = torch.argmax(action_probs, dim=1).item()  # Select the action with the highest probability
      return action

    def update(self, batch, agent_id, agents, gamma=0.99, tau=0.01):
        states, actions, rewards, next_states, dones = batch

        # Get the number of agents
        n_agents = len(agents)

        # Get the current agent's state and action
        state = states[agent_id]
        action = actions[agent_id]

        # Get the next state for the current agent
        next_state = next_states[agent_id]

        # Get the target actions for the next states using the target actor networks
        next_actions = []
        for i in range(n_agents):
            next_action = agents[i].target_actor(next_states[i])
            next_actions.append(next_action)
        next_actions = torch.cat(next_actions, dim=1)

        # Compute the target Q-value using the target critic network
        target_q = agents[agent_id].target_critic(next_state, next_actions)
        target_q = rewards[agent_id] + (gamma * target_q * (1 - dones[agent_id]))

        # Compute the current Q-value using the critic network
        current_q = agents[agent_id].critic(state, actions)

        # Compute the critic loss using the TD error (Q-targets - Q-values)
        critic_loss = F.mse_loss(current_q, target_q)

        # Update the critic network by minimizing the critic loss
        agents[agent_id].critic_optimizer.zero_grad()
        critic_loss.backward()
        agents[agent_id].critic_optimizer.step()

        # Compute the actor loss using the critic network and the current actions
        actor_loss = -agents[agent_id].critic(state, actions).mean()

        # Update the actor network by minimizing the actor loss
        agents[agent_id].actor_optimizer.zero_grad()
        actor_loss.backward()
        agents[agent_id].actor_optimizer.step()

        # Perform soft updates of the target networks for both actor and critic
        for target_param, param in zip(agents[agent_id].target_actor.parameters(), agents[agent_id].actor.parameters()):
            target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)

        for target_param, param in zip(agents[agent_id].target_critic.parameters(), agents[agent_id].critic.parameters()):
            target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)

In [148]:


class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        experience = (state, action, reward, next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        experiences = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*experiences)
        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.buffer)

In [152]:

class ForagingEnv(AECEnv):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super().__init__()
        self.grid_size = 3
        self.n_agents = 2
        self.observation_size = self.grid_size * self.grid_size * 3 + 2
        self.action_size = 6
        self.agents = ['agent_' + str(i) for i in range(self.n_agents)]  # Agent identifiers
        self.agent_selector = agent_selector(self.agents)
        self.action_spaces = {agent: spaces.Discrete(6) for agent in self.agents}  # Add one for "communicate" action
        self.observation_spaces = {agent: spaces.Dict({
            "grid": spaces.Box(low=0, high=2, shape=(self.grid_size, self.grid_size, 3), dtype=np.float32),
            "state": spaces.Dict({
                "position": spaces.MultiDiscrete([self.grid_size, self.grid_size]),
                "water_timer": spaces.Discrete(3),  # Timesteps until death without water
                "food_timer": spaces.Discrete(7),  # Timesteps until death without food
                "beliefs": spaces.Dict({
                    "water_replenish_rate": spaces.Discrete(10),  # Example max rate
                    "food_replenish_rate": spaces.Discrete(10),
                })
            })
        }) for agent in self.agents}


        self.grid = np.zeros((self.grid_size, self.grid_size, 3), dtype=np.float32)  # Third dimension for agent presence, water, food
        self.resource_counters = {"water": [np.inf, 0], "food": [np.inf, 0]}  # [acquisitions left, replenishment timer]
        self.agent_states = {agent: {"position": None, "water_timer": 3, "food_timer": 7, "beliefs": {"water_replenish_rate": np.inf, "food_replenish_rate": np.inf}} for agent in self.agents}
        self.current_agent = None
        self.messages = {agent: "" for agent in self.agents}  # Initialize messages for each agent
        # Initialize environment state variables

    def reset(self):
      self.agent_selector.reinit(self.agents)
      self.current_agent = self.agent_selector.next()
      self.grid *= 0  # Clear the grid

      # Randomly place water and food, initialize resource counters
      water_position = np.random.choice(self.grid_size**2)
      food_position = np.random.choice(self.grid_size**2)
      while food_position == water_position:
          food_position = np.random.choice(self.grid_size**2)

      self.grid[water_position // self.grid_size, water_position % self.grid_size, 1] = 1
      self.grid[food_position // self.grid_size, food_position % self.grid_size, 2] = 1
      self.resource_counters["water"] = [np.random.randint(1, 5), np.random.randint(5, 15)]  # Random example values
      self.resource_counters["food"] = [np.random.randint(1, 5), np.random.randint(5, 15)]

      # Set initial positions for agents and reset their states
      for i, agent_id in enumerate(self.agents):
          while True:
              pos = np.random.choice(self.grid_size**2)
              if self.grid[pos // self.grid_size, pos % self.grid_size].sum() == 0:  # Ensure the position is empty
                  self.grid[pos // self.grid_size, pos % self.grid_size, 0] = i + 1  # Mark agent's presence
                  agent = self.agents[i]  # Access agent by identifier
                  self.agent_states[agent] = self.agent_states.get(agent, {})  # Ensure agent state exists
                  self.agent_states[agent]["position"] = (pos // self.grid_size, pos % self.grid_size)
                  self.agent_states[agent]["water_timer"] = 3
                  self.agent_states[agent]["food_timer"] = 7
                  break

      # Return the initial observation for each agent using agent IDs as keys
      return {agent_id: self.observe(agent_id) for agent_id in self.agents}

    def step(self, actions):
        rewards = {}
        done = False
        info = {}  # Additional info, if any

        # Reset any necessary state (e.g., messages) at the start of a step
        self.messages = {agent_id: "" for agent_id in self.agents}

        for agent_id in self.agents:
            # Ensure action is fetched using the agent_id
            action_value = actions[agent_id]

            reward = 0  # Initialize reward for this step

            # Movement, consumption, or communication based on action_value
            if action_value < 4:  # Movement actions
                move_success = self.move_agent(agent_id, action_value)
                reward -= 1 if move_success else 5  # Penalty for unsuccessful move
            elif action_value == 4:  # Consumption action
                reward += self.consume_resources(agent_id)
            elif action_value == 5:  # Communication action
                communicated_successfully = self.communicate(agent_id)
                reward += 2 if communicated_successfully else -2  # Reward/penalty based on communication

            # Update survival timers and check for termination conditions
            self.agent_states[agent_id]["water_timer"] -= 1
            self.agent_states[agent_id]["food_timer"] -= 1
            if self.agent_states[agent_id]["water_timer"] <= 0 or self.agent_states[agent_id]["food_timer"] <= 0:
                self.terminate_agent(agent_id)
                reward -= 50  # Penalty for agent termination

            # Survival reward
            reward += 0.1

            rewards[agent_id] = reward

        done = len(self.agents) == 0  # Check if episode is done (all agents terminated)

        # Construct return values using agent identifiers for keys
        observations = {agent_id: self.observe(agent_id) for agent_id in self.agents}
        return observations, rewards, done, info

    def move_agent(self, agent, direction):
        pos = self.agent_states[agent]["position"]
        move_success = False  # Assume move is unsuccessful by default
        if direction == 0:  # Up
            new_pos = (max(pos[0] - 1, 0), pos[1])
        elif direction == 1:  # Down
            new_pos = (min(pos[0] + 1, self.grid_size - 1), pos[1])
        elif direction == 2:  # Left
            new_pos = (pos[0], max(pos[1] - 1, 0))
        else:  # Right
            new_pos = (pos[0], min(pos[1] + 1, self.grid_size - 1))

        # Update position if the new position is not occupied
        if self.grid[new_pos[0], new_pos[1], 0] == 0:
            self.grid[pos[0], pos[1], 0] = 0  # Remove agent from old position
            self.grid[new_pos[0], new_pos[1], 0] = 1  # Add agent to new position
            self.agent_states[agent]["position"] = new_pos
            move_success = True  # The move was successful
        return move_success

    def consume_resources(self, agent):
      pos = self.agent_states[agent]["position"]
      reward = 0
      # Check for water
      if self.grid[pos[0], pos[1], 1] == 1:
          # Consume water if available
          if self.resource_counters["water"][0] > 0:  # If water is available
              self.resource_counters["water"][0] -= 1  # Decrement water availability
              self.agent_states[agent]["water_timer"] = 3  # Reset water timer
              reward += 10  # Reward for consuming water
              if self.resource_counters["water"][0] <= 0:  # If water is now depleted
                  self.grid[pos[0], pos[1], 1] = 0  # Remove water from the grid
      # Check for food
      elif self.grid[pos[0], pos[1], 2] == 1:
          # Consume food if available
          if self.resource_counters["food"][0] > 0:  # If food is available
              self.resource_counters["food"][0] -= 1  # Decrement food availability
              self.agent_states[agent]["food_timer"] = 7  # Reset food timer
              reward += 10  # Reward for consuming food
              if self.resource_counters["food"][0] <= 0:  # If food is now depleted
                  self.grid[pos[0], pos[1], 2] = 0  # Remove food from the grid
      return reward

    def communicate(self, sender):
      pos = self.agent_states[sender]["position"]
      water_timer = self.agent_states[sender]["water_timer"]
      food_timer = self.agent_states[sender]["food_timer"]
      water_belief = self.agent_states[sender]["beliefs"]["water_replenish_rate"]
      food_belief = self.agent_states[sender]["beliefs"]["food_replenish_rate"]

      # Determine the urgency level for water and food based on remaining timers
      water_urgency = "high" if water_timer <= 2 else "low"
      food_urgency = "high" if food_timer <= 3 else "low"

      message = {
          "location": pos,
          "water_belief": water_belief,
          "food_belief": food_belief,
          "water_urgency": water_urgency,
          "food_urgency": food_urgency,
          "found": None
      }

      communicated_successfully = False
      # Check for the presence of water or food at the sender's location
      if self.grid[pos[0], pos[1], 1] == 1:  # Water found
          message["found"] = "water"
          communicated_successfully = True
      elif self.grid[pos[0], pos[1], 2] == 1:  # Food found
          message["found"] = "food"
          communicated_successfully = True

      # If something was found, broadcast the message
      if communicated_successfully:
          for agent in self.agents:
              if agent != sender:
                  self.messages[agent] = message

      return communicated_successfully

    def terminate_agent(self, agent):
        # logic for handling agent termination
        self.agents.remove(agent)  # Remove the agent from the active list
        self.grid[self.agent_states[agent]["position"][0], self.agent_states[agent]["position"][1], 0] = 0  # Clear the agent from the grid
        del self.agent_states[agent]  # Remove the agent's state


    def observe(self, agent):
      # Return agent-specific observations including both grid and their internal state
      observation = self.grid.copy()
      agent_state = self.agent_states[agent]
      observed_message = self.messages[agent]

      # If there's a message, update beliefs based on the message
      if observed_message:
          if observed_message["found"] == "water":
              agent_state["beliefs"]["water_replenish_rate"] = observed_message["water_belief"]
          elif observed_message["found"] == "food":
              agent_state["beliefs"]["food_replenish_rate"] = observed_message["food_belief"]

      return {"grid": observation, "state": agent_state, "message": observed_message}

    def update_resources(self):
      # Iterate through each resource to update its status
      for resource, counter in self.resource_counters.items():
          if counter[0] <= 0:  # If depleted
              counter[1] -= 1  # Decrement replenishment timer
              if counter[1] <= 0:  # If it's time to replenish
                  self.replenish_resource(resource)

    def replenish_resource(self, resource):
      # Randomly choose a new position for the resource
      position = np.random.choice(self.grid_size**2)
      if resource == "water":
          self.grid[position // self.grid_size, position % self.grid_size, 1] = 1  # Place water
          self.resource_counters["water"] = [np.random.randint(1, 5), np.random.randint(5, 15)]  # Reset counters
      elif resource == "food":
          self.grid[position // self.grid_size, position % self.grid_size, 2] = 1  # Place food
          self.resource_counters["food"] = [np.random.randint(1, 5), np.random.randint(5, 15)]  # Reset counters

    def render(self, mode="human"):
        # Visualize the current state of the environment, including agent positions, resources, and timers
        if mode == "human":
          for r in range(self.grid_size):
              print("+---" * self.grid_size + "+")
              for c in range(self.grid_size):
                  cell = " "
                  if self.grid[r, c, 0] > 0:  # Agent presence
                      cell = "A"
                  elif self.grid[r, c, 1] == 1:  # Water
                      cell = "W"
                  elif self.grid[r, c, 2] == 1:  # Food
                      cell = "F"
                  print(f"| {cell} ", end="")
              print("|")
          print("+---" * self.grid_size + "+")
        elif mode == "rgb_array":
          self.ax.clear()
          self.ax.axis('off')

          # Draw the grid
          for r in range(self.grid_size):
              for c in range(self.grid_size):
                  if self.grid[r, c, 0] > 0:  # Agent presence
                      self.ax.text(c, r, 'A', fontsize=12, ha='center', va='center', color='blue')
                  elif self.grid[r, c, 1] == 1:  # Water
                      self.ax.text(c, r, 'W', fontsize=12, ha='center', va='center', color='cyan')
                  elif self.grid[r, c, 2] == 1:  # Food
                      self.ax.text(c, r, 'F', fontsize=12, ha='center', va='center', color='green')

          # Draw grid lines
          self.ax.set_xticks(np.arange(-0.5, self.grid_size, 1), minor=True)
          self.ax.set_yticks(np.arange(-0.5, self.grid_size, 1), minor=True)
          self.ax.grid(which="minor", color="w", linestyle='-', linewidth=2)
          self.ax.tick_params(which="minor", size=0)

          # Update the display
          self.fig.canvas.draw()
          plt.pause(0.01)  # Small delay to allow for real-time updating




In [160]:
# Example action selection process
actions = {agent_id: env.action_spaces[agent_id].sample() for agent_id in env.agents}
print(f"Actions: {actions}")

Actions: {<__main__.MADDPGAgent object at 0x7de92cf31420>: 4, <__main__.MADDPGAgent object at 0x7de9428dca30>: 1}


In [159]:
env = ForagingEnv()
state = env.reset()
print(f"Initial State: {state}")

done = False
while not done:
    actions = {agent: env.action_spaces[agent].sample() for agent in env.agents}  # Random actions for demonstration
    print(f"Actions: {actions}")
    next_state, rewards, done, info = env.step(actions)
    print(f"Next State: {next_state}, Rewards: {rewards}, Done: {done}")

Initial State: {0: {'grid': array([[[0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 1.],
        [2., 0., 0.]]], dtype=float32), 'state': {'position': (1, 2), 'water_timer': 3, 'food_timer': 7, 'beliefs': {'water_replenish_rate': inf, 'food_replenish_rate': inf}}, 'message': ''}, 1: {'grid': array([[[0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 1.],
        [2., 0., 0.]]], dtype=float32), 'state': {'position': (2, 2), 'water_timer': 3, 'food_timer': 7, 'beliefs': {'water_replenish_rate': inf, 'food_replenish_rate': inf}}, 'message': ''}}
Actions: {<__main__.MADDPGAgent object at 0x7de92cf31420>: 5, <__main__.MADDPGAgent object at 0x7de9428dca30>: 5}


TypeError: '<' not supported between instances of 'MADDPGAgent' and 'int'

In [155]:
# Initialize environment
env = ForagingEnv()

# Initialize agents with their actor and critic networks
agents = [MADDPGAgent(obs_size=env.grid_size * env.grid_size * 3, action_size=env.action_size, n_agents=env.n_agents) for _ in range(env.n_agents)]

obs = env.reset()
done = False
while not done:
    # Assume agents can select actions based on observations
    actions = [agents[agent_id].act(obs[agent]) for agent_id, agent in enumerate(env.agents)]
    next_obs, rewards, done, _ = env.step(actions)
    # Store (obs, actions, rewards, next_obs) in replay buffer
    # Sample a batch from the replay buffer and update both actor and critic models
    obs = next_obs

KeyError: <__main__.MADDPGAgent object at 0x7de92d1f2590>

In [ ]:
# Initialize environment
env = ForagingEnv()

# Initialize agents with their actor and critic networks
agents = [MADDPGAgent(obs_size=env.observation_size, action_size=env.action_size, n_agents=env.n_agents) for _ in range(env.n_agents)]

# Initialize replay buffer
replay_buffer = ReplayBuffer(capacity=10000)

num_episodes = 100  # Specify the number of episodes

for episode in range(num_episodes):
    state = env.reset()
    done = False
    while not done:
        actions = [agent.act(state[agent_id]) for agent_id, agent in enumerate(agents)]
        next_state, rewards, done, _ = env.step(actions)

        # Store experience in replay buffer
        replay_buffer.add(state, actions, rewards, next_state, done)

        # Sample a batch of experiences from the buffer
        batch = replay_buffer.sample(batch_size=64)  # Specify the batch size

        # Update each agent - this involves updating both the actor and critic networks
        for agent_id, agent in enumerate(agents):
            agent.update(batch, agent_id, agents)

        state = next_state

        if done:
            break